# Visualizing KMeans ++ 

### Debanuj | Shivansh | Kunal | Mridul | Anmol

In [28]:
import ipywidgets as iwg
import matplotlib.pyplot as plt
import random
import numpy as np
plt.rcParams.update({'font.size': 17})


## Data Generation Function

In [29]:
points =[]
def generate_data(n_points, n_clusters,var):
    global points
    mu = []
    points = np.zeros((n_points,3))
    for i in range(n_clusters):
        mu.append([random.randint(0,1000),random.randint(0,1000)])
    for i in range(n_points):
        j = random.randint(0,n_clusters-1)
        points[i][0] = random.gauss(mu[j][0],var)
        points[i][1] = random.gauss(mu[j][1],var)
        points[i][2] = j
    
    plt.figure(figsize=(10,10))
    plt.title("Generated Data Points")
    plt.xlabel("X - axis")
    plt.ylabel("Y - axis")
    plt.scatter(points[:,0],points[:,1],c = points[:,2])


In [30]:
print("Drag the sliders to generate data as required")
quater = iwg.interact(generate_data, n_points=iwg.IntSlider(min=100,max=500,step=50,value=300),n_clusters=iwg.IntSlider(min=2,max=10,step=1,value=3),var=iwg.IntSlider(min=1,max=200,step=20,value=45));

Drag the sliders to generate data as required


interactive(children=(IntSlider(value=300, description='n_points', max=500, min=100, step=50), IntSlider(value…

## KMeans ++ Initializer Function

In [31]:
def dis(a,b,X):
    return((X[a][0]-X[b][0])**2+(X[a][1]-X[b][1])**2)

def kmpp(X,k):
    # k is number of centers
    # X is the data
    curr_points = []
    n = X.shape[0]
    curr_points.append(random.randint(0,n-1))
    
    for i in range(k-1):
      #  print(curr_points)
        prob = np.zeros((n,1))
        mini = float('inf')
        for j in range(n):
            for k in curr_points:
                temp = dis(k,j,X)
                if(temp<mini):
                    mini = temp
                    prob[j] = mini
        prob = prob/sum(prob)
        curr_points.append(np.random.choice(n,1,list(prob))[0])
        centers = np.array([X[i] for i in curr_points])
    
    return centers

## Updater Function

In [32]:
def Sq_Dist(a,b):
    return((a[0]-b[0])**2 +(a[1]-b[1])**2)

def Kmeans_Update_cluster(data,Centroids):
   #Reassigning the data points to the centroids
    cluster_Data={}
    color =[]
    Centroids = list(Centroids)
    
    for i in range(len(data)):
        x =data[i]
        minimum = Sq_Dist(x,Centroids[0])
        centerID=0
        for j in range(1,len(Centroids)):
            dist = Sq_Dist(x,Centroids[j])
            if(dist<minimum):
                minimum = dist
                centerID = j
                
        color.append(centerID + 1)       
        if(centerID not in cluster_Data.keys()):
            cluster_Data[centerID] =[i]
        else:
            cluster_Data[centerID].append(i)
        
    return cluster_Data,color
       
def Kmeans_Update_centroids(data,cluster_Data):
    # Calculating new centroids based on the clusters found
    centroids=[]
    for key in cluster_Data.keys():

        cluster_points_x = [data[j][0] for j in cluster_Data[key]]
        cluster_points_y = [data[j][1] for j in cluster_Data[key]]

        x_coor = sum(cluster_points_x)/len(cluster_Data[key])
        y_coor = sum(cluster_points_y)/len(cluster_Data[key])

        centroids.append([x_coor,y_coor])

    return np.array(centroids)

## Functon for calculation of Residual Sum of Squares 

In [33]:
def rss_clusters(points,cd,color,centers):
    total = 0
    for key in cd.keys():
        center = centers[key]
        for i in cd[key]:
            total+= Sq_Dist(center,points[i])
    return total
   

## Initializing KMeans ++

In [34]:
flag1 = 1
itr = 0

def kmeans_initialize(n_centers,state):
    global itr
    global flag1
    global centers
    if(flag1 != state):
        itr = 0
        centers = kmpp(points,n_centers)
        flag1 = (flag1+1)%2
        #print(centers.shape) 
        plt.figure(figsize=(10,10))
        plt.title("Initialized Clusters and Centers")
        plt.xlabel("X - axis")
        plt.ylabel("Y - axis")
        plt.scatter(points[:,0],points[:,1],c = ['c']*points.shape[0])
        plt.scatter(centers[:,0],centers[:,1],c = ['r']*len(centers))
        plt.show()
        
    


In [35]:
print("Drag the sliders to initialize KMeans ++ as required")
semi = iwg.interact(kmeans_initialize, n_centers=iwg.IntSlider(min=2,max=10,step=1,value=3),state=iwg.IntSlider(min=0,max=1,step=1,value = 1))

Drag the sliders to initialize KMeans ++ as required


interactive(children=(IntSlider(value=3, description='n_centers', max=10, min=2), IntSlider(value=1, descripti…

## Visualizing Iterations

In [36]:
flag2 = 1
def kmeans_iteration(state):
    
    global itr
    global flag2
    global centers
    if(flag2!=state):
        itr=itr +1
        flag2 = (flag2+1)%2
        cd,color = Kmeans_Update_cluster(points,centers)
        
        try:
            score = rss_clusters(points,cd,color,centers)
        except:
            score= 0         
        
        centers = Kmeans_Update_centroids(points,cd)
        plt.figure(figsize=(10,10))
        plt.xlabel("X - axis")
        plt.ylabel("Y - axis")
        plt.title("Kmeans Iteration no : " + str(itr) + ' RSS score = %d'%(score))
        plt.scatter(points[:,0],points[:,1],c = color,s=20,alpha = 0.3)
        plt.scatter(centers[:,0],centers[:,1],c = ['k']*len(centers),s= 100,marker = '*')
        plt.show()
    

In [37]:
print(" Drag the slider to switch the state and observe the next iteration ")
final = iwg.interact(kmeans_iteration, state=iwg.IntSlider(min=0,max=1,step=1,value = 1))

 Drag the slider to switch the state and observe the next iteration 


interactive(children=(IntSlider(value=1, description='state', max=1), Output()), _dom_classes=('widget-interac…